In [155]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from moviepy.editor import VideoClip, ImageClip, concatenate_videoclips
import pandas as pd

runnersite = "https://runalyze.com/athlete/Schrottie"

In [156]:
def extract_activity_data():
    response = requests.get(runnersite)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    activities = []
    today = datetime.today().strftime('%d.%m.%Y')
    
    # Finden aller Zeilen mit Laufaktivitäten
    activity_rows = soup.find_all('tr', class_='r')
    
    data = {
        'date': [],
        'a_type': [],
        'r_type': [],
        'distance': [],
        'duration': [],
        'pace': []
    }
    
    current_date = None
    skipped_datecell = False
    
    for row in activity_rows:
        cols = row.find_all('td')
        
        if cols:
            date_cell = cols[1].text.strip()
            if date_cell:
                new_date = date_cell.split()[0].replace('.', '') + '.' + str(datetime.today().year) + '.' # Korrektur des Datumsformats
                
                # Überprüfen, ob das Datum in der aktuellen Zeile nicht leer ist und sich vom vorherigen Datum unterscheidet
                if new_date and new_date != current_date:
                    current_date = new_date
                    skipped_datecell = False
                if new_date and new_date == current_date:   
                    skipped_datecell = True
                    
            if row.find('i', class_='icons8-Running'):
                if new_date and skipped_datecell:
                    # Aktualisieren der Indizes für die Zellen, wenn das Datum aus der Variablen current_date genommen wird
                    rt_str = cols[4].text.strip() if len(cols) > 4 else ''
                    distance_str = cols[5].text.strip().split()[0].replace(',', '.') if len(cols) > 5 else ''
                    duration_str = cols[6].text.strip() if len(cols) > 6 else ''
                    pace_str = cols[7].text.strip() if len(cols) > 7 else ''
                else:
                    rt_str = cols[3].text.strip() if len(cols) > 3 else ''
                    distance_str = cols[4].text.strip().split()[0].replace(',', '.') if len(cols) > 4 else ''
                    duration_str = cols[5].text.strip() if len(cols) > 5 else ''
                    pace_str = cols[6].text.strip() if len(cols) > 6 else ''
                
                # Überprüfen, ob die aktuellen Werte nicht leer sind
                if distance_str and duration_str and pace_str:
                    activities.append({'date': current_date, 'a_type': 'run', 'r_type': rt_str, 'distance': distance_str, 'duration': duration_str, 'pace': pace_str})
                else:
                    # Verwenden des vorherigen Datums, wenn die aktuellen Werte leer sind
                    if data['date']:
                        activities.append({'date': data['date'][-1], 'a_type': 'run', 'r_type': rt_str, 'distance': distance_str, 'duration': duration_str, 'pace': pace_str})
    
    df = pd.DataFrame(activities)
    df = df[df['a_type'] == 'run'] # Filtern nach Aktivitätstyp "run"
    return df


df = extract_activity_data()
print(df)

          date a_type r_type distance  duration     pace
0   0103.2024.    run              ER  10,94 km  1:10:15
1   0203.2024.    run              ER  10,19 km  1:00:00
2   0303.2024.    run              ER  35,68 km  3:31:29
3   0503.2024.    run     ER    30.05   2:58:59  5:57/km
4   0703.2024.    run     ER    27.34   2:44:34  6:01/km
5   0803.2024.    run              ER  10,53 km  1:05:48
6   0903.2024.    run              ER  14,09 km  1:30:34
7   1003.2024.    run              ER  28,56 km  2:52:17
8   1203.2024.    run     ER    30.14   2:53:41  5:46/km
9   1403.2024.    run     ER    30.22   2:57:15  5:52/km
10  1503.2024.    run              ER  13,21 km  1:24:20
11  1603.2024.    run              ER  11,28 km  1:12:24
12  1703.2024.    run              ER  42,79 km  4:17:13


In [157]:
def main():
    activities = extract_activity_data()
    if not activities.empty:
        print("Gefundene Aktivitäten:")
        print(activities)
    else:
        print("Keine Aktivitäten gefunden.")

if __name__ == "__main__":
    main()


Gefundene Aktivitäten:
          date a_type r_type distance  duration     pace
0   0103.2024.    run              ER  10,94 km  1:10:15
1   0203.2024.    run              ER  10,19 km  1:00:00
2   0303.2024.    run              ER  35,68 km  3:31:29
3   0503.2024.    run     ER    30.05   2:58:59  5:57/km
4   0703.2024.    run     ER    27.34   2:44:34  6:01/km
5   0803.2024.    run              ER  10,53 km  1:05:48
6   0903.2024.    run              ER  14,09 km  1:30:34
7   1003.2024.    run              ER  28,56 km  2:52:17
8   1203.2024.    run     ER    30.14   2:53:41  5:46/km
9   1403.2024.    run     ER    30.22   2:57:15  5:52/km
10  1503.2024.    run              ER  13,21 km  1:24:20
11  1603.2024.    run              ER  11,28 km  1:12:24
12  1703.2024.    run              ER  42,79 km  4:17:13
